In [1]:
from IPython.display import display, HTML
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.appName("GameOfThronesAnalysis").getOrCreate()

def display_df(df, rows=1):
    display(HTML(df.toPandas().head(rows).to_html()))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/20 18:06:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# dataset = spark.read.csv("../medium_search_dataset.csv", header=True, inferSchema=True)
# print("dataset rows:", dataset.count())
# data = dataset.select('post_time', 'post_name', 'ranking', 'Searched')
data = spark.read.csv("../got_scripts_breakdown.csv", header=True, inferSchema=True)
print("data rows:", data.count())
data.describe()

df_agg = data.orderBy('Name', 'N_series').groupBy('Name').agg(F.array_join(F.collect_list("Sentence"), ' ', ' ').alias("Dialogue"))
display_df(df_agg, 1)


data rows: 22510


,Name,Dialogue
0,None,"Do the Dothrakis buy their slaves? I didn’t realize there were knights of the Blackwater Lyanna Would you get out of here? We’re sparring I don’t know But he’s got giant’s blood Winter is here, Your Grace We need the King in the North in the north There is a snow blown field ARYA is riding on a horse Winterfell can be seen far in the distance"


In [3]:
import string
from nltk.stem.snowball import SnowballStemmer
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.types import ArrayType, StringType


tokenizer = Tokenizer(inputCol="Dialogue", outputCol="words")
token_data = tokenizer.transform(df_agg)

stopwords = StopWordsRemover.loadDefaultStopWords('english')
stopwords.extend(['it'])
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=stopwords)
# print(stopwords_remover.getStopWords())
df_no_stopwords = stopwords_remover.transform(token_data)


def clean_stem(stemmer, token):
    """ Return stem word, removing any punctuation from end of string """
    t = stemmer.stem(token)
    return t.rstrip(string.punctuation) 
    # return t.translate(str.maketrans('', '', string.punctuation))
    
stemmer = SnowballStemmer(language='english')
stemmer_udf = F.udf(lambda tokens: [clean_stem(stemmer, token) for token in tokens], ArrayType(StringType()))
df_stemmed = df_no_stopwords.withColumn("words_stemmed", stemmer_udf("filtered_words"))


cv = CountVectorizer(inputCol="words_stemmed", outputCol="features")
cv_model = cv.fit(df_stemmed)
vocabulary = {i:v for i,v in enumerate(cv_model.vocabulary)}
vector_data = cv_model.transform(df_stemmed)
display_df(vector_data, 5)

,Name,Dialogue,words,filtered_words,words_stemmed,features
0,None,"Do the Dothrakis buy their slaves? I didn’t realize there were knights of the Blackwater Lyanna Would you get out of here? We’re sparring I don’t know But he’s got giant’s blood Winter is here, Your Grace We need the King in the North in the north There is a snow blown field ARYA is riding on a horse Winterfell can be seen far in the distance","[do, the, dothrakis, buy, their, slaves?, i, didn’t, realize, there, were, knights, of, the, blackwater, lyanna, would, you, get, out, of, here?, we’re, sparring, i, don’t, know, but, he’s, got, giant’s, blood, winter, is, here,, your, grace, we, need, the, king, in, the, north, in, the, north, there, is, a, snow, blown, field, arya, is, riding, on, a, horse, winterfell, can, be, seen, far, in, the, distance]","[dothrakis, buy, slaves?, didn’t, realize, knights, blackwater, lyanna, get, here?, we’re, sparring, don’t, know, he’s, got, giant’s, blood, winter, here,, grace, need, king, north, north, snow, blown, field, arya, riding, horse, winterfell, seen, far, distance]","[dothraki, buy, slaves, didn't, realiz, knight, blackwat, lyanna, get, here, we'r, spar, don't, know, he, got, giant, blood, winter, here, grace, need, king, north, north, snow, blown, field, arya, ride, hors, winterfel, seen, far, distanc]","(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...)"
1,a voice,"It’s Maester Luwin, my lord","[it’s, maester, luwin,, my, lord]","[it’s, maester, luwin,, lord]","[it, maester, luwin, lord]","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
2,a younger melara,We shouldn’t be out here alone,"[we, shouldn’t, be, out, here, alone]","[shouldn’t, alone]","[shouldn't, alon]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
3,addam marbrand,ls it true about Stannis and Renly?,"[ls, it, true, about, stannis, and, renly?]","[ls, true, stannis, renly?]","[ls, true, stanni, renly]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...)"
4,aemon,"And what is it that couldnt wait until morning, Tarly? That it was you? Who else but Samwell Tarly would be wasting candles to read in the middle of the night? Oh, no need I know my way around this 

In [ ]:
from pyspark.ml.clustering import LDA

lda = LDA(k=15, maxIter=30, featuresCol="features")
lda_model = lda.fit(vector_data)

topics = lda_model.describeTopics(3)


In [ ]:
import pyspark.sql.functions as F

map_array = F.udf(lambda a: [vocabulary[k] for k in a])
df = topics.withColumn('terms', map_array(topics['termIndices']))
display_df(df, 50)